### 1.0 Libraries and directories

In [11]:
import glob
import pprint as pp
import pandas as pd
import geopandas as gpd
import ee
import geemap

ee.Authenticate()
ee.Initialize()

rois = ee.data.listAssets('projects/alpod-412314/assets/ROIs/')
rois_ids = [roi['id'] for roi in rois['assets']]
rois_ids.remove('projects/alpod-412314/assets/ROIs/all_regions')
pp.pp(rois_ids)


['projects/alpod-412314/assets/ROIs/AKCP_roi',
 'projects/alpod-412314/assets/ROIs/MRD_TUK_Anderson_roi',
 'projects/alpod-412314/assets/ROIs/MRD_roi',
 'projects/alpod-412314/assets/ROIs/TUK_roi',
 'projects/alpod-412314/assets/ROIs/YKdelta_roi',
 'projects/alpod-412314/assets/ROIs/YKflats_roi',
 'projects/alpod-412314/assets/ROIs/anderson_plain_roi']


## !! 2.0 Run these for Monthly Recurrence (all years)

In [2]:
# rois = ee.data.listAssets('projects/alpod-412314/assets/ROIs/')
# rois_ids = [roi['id'] for roi in rois['assets']]
# rois_ids.remove('projects/alpod-412314/assets/ROIs/all_regions')
# pp.pp(rois_ids)


# gswo_monthly_recurrence = ee.ImageCollection('JRC/GSW1_4/MonthlyRecurrence')
# june = 6
# aug = 8
# june_data = gswo_monthly_recurrence.filter(ee.Filter.eq('month', june)).first()
# aug_data = gswo_monthly_recurrence.filter(ee.Filter.eq('month', aug)).first()

['projects/alpod-412314/assets/ROIs/AKCP_roi',
 'projects/alpod-412314/assets/ROIs/MRD_TUK_Anderson_roi',
 'projects/alpod-412314/assets/ROIs/MRD_roi',
 'projects/alpod-412314/assets/ROIs/TUK_roi',
 'projects/alpod-412314/assets/ROIs/YKdelta_roi',
 'projects/alpod-412314/assets/ROIs/YKflats_roi',
 'projects/alpod-412314/assets/ROIs/anderson_plain_roi']


In [3]:
# def export_rois(roi_id, target_img, month_name):
    
#     roi_name_temp = roi_id.split('/')[-1]
#     roi_name = roi_name_temp.split('_roi')[0]
#     feature = ee.FeatureCollection(roi_id).first()
#     bounds = feature.geometry()

#     clipped_img = target_img.clip(bounds)

#     export_params = {
#         'image': clipped_img,
#         'description': f'GSWORecurrence_{roi_name}_month_{month_name}_', 
#         'fileNamePrefix': f'GSWORecurrence_{roi_name}_month_{month_name}_',
#         'folder': 'gswo_exports', 
#         'scale': 30,
#         'crs': 'EPSG:4326',
#         'region': bounds,
#         'fileFormat': 'GeoTIFF',
#         'maxPixels': 1e13
#     }

#     ee.batch.Export.image.toDrive(**export_params).start()

In [4]:
# datasets = [june_data, aug_data]
# month_names = ['june', 'aug']

# monthly_datasets = list(zip(month_names, datasets))

# for month, ds in monthly_datasets:
#     for roi_id in rois_ids:
#         export_rois(roi_id, ds, month)

## !! Run these for Monthly History (specific years)

In [12]:
gswo_monthly_water_hist = ee.ImageCollection('JRC/GSW1_4/MonthlyHistory')
years = range(1984, 2022) # 2016 through 2020 is common timeframe | 1984 through 2021 is full timeframe

june_idx = [f'{year}_06' for year in years]
print(june_idx)
aug_idx = [f'{year}_08' for year in years]
print(aug_idx)

june_imgs = gswo_monthly_water_hist.filter(ee.Filter.inList('system:index', june_idx))
aug_imgs = gswo_monthly_water_hist.filter(ee.Filter.inList('system:index', aug_idx))

june_max_obs = june_imgs.size().getInfo()
aug_max_obs = aug_imgs.size().getInfo()
print(june_max_obs, aug_max_obs)

['1984_06', '1985_06', '1986_06', '1987_06', '1988_06', '1989_06', '1990_06', '1991_06', '1992_06', '1993_06', '1994_06', '1995_06', '1996_06', '1997_06', '1998_06', '1999_06', '2000_06', '2001_06', '2002_06', '2003_06', '2004_06', '2005_06', '2006_06', '2007_06', '2008_06', '2009_06', '2010_06', '2011_06', '2012_06', '2013_06', '2014_06', '2015_06', '2016_06', '2017_06', '2018_06', '2019_06', '2020_06', '2021_06']
['1984_08', '1985_08', '1986_08', '1987_08', '1988_08', '1989_08', '1990_08', '1991_08', '1992_08', '1993_08', '1994_08', '1995_08', '1996_08', '1997_08', '1998_08', '1999_08', '2000_08', '2001_08', '2002_08', '2003_08', '2004_08', '2005_08', '2006_08', '2007_08', '2008_08', '2009_08', '2010_08', '2011_08', '2012_08', '2013_08', '2014_08', '2015_08', '2016_08', '2017_08', '2018_08', '2019_08', '2020_08', '2021_08']
38 38


In [13]:
"""GSWO bitmask for water is...
0: no data
1: no water 
2: water
"""
def select_invalid_pixels(img, mask_val):

    img_unmasked = img.unmask(mask_val)
    inval_pixels = img_unmasked.eq(mask_val)
    inval_pixels = inval_pixels.updateMask(ee.Image.constant(1))
    return inval_pixels

def select_water_pixels(img, water_val):
    water_pixels = img.eq(water_val)
    return water_pixels

In [14]:
def mask_invalid_pixels(img, mask_val):
    data_mask = img.neq(mask_val)
    data = img.updateMask(data_mask)
    return data

In [15]:
def calc_valid_occurrence(img_collection, max_obs, mask_val, water_val):
    # 
    invalid_masks = img_collection.map(lambda img: select_invalid_pixels(img, mask_val))
    sum_invalid = invalid_masks.reduce(ee.Reducer.sum())

    # Sum valid water obsergations in each image
    data = img_collection.map(lambda img: mask_invalid_pixels(img, mask_val))
    wtr_imgs = data.map(lambda img: select_water_pixels(img, water_val))
    sum_water = wtr_imgs.reduce(ee.Reducer.sum())

    # Calculate the % of valid water occurence. 
    max_obs_img = ee.Image.constant(max_obs)
    valid_obs = max_obs_img= max_obs_img.subtract(sum_invalid)
    valid_occurrence = sum_water.divide(valid_obs)

    return valid_occurrence
    

In [16]:
aug_occurrence = calc_valid_occurrence(aug_imgs, aug_max_obs, mask_val=0, water_val=2)
june_occurrence = calc_valid_occurrence(june_imgs, june_max_obs, mask_val=0, water_val=2)

month_names = ['june', 'aug']
datasets = [june_occurrence, aug_occurrence]

monthly_datasets = list(zip(month_names, datasets))


In [8]:
Map = geemap.Map(center=[0, 0], zoom=2)
Map.addLayer(aug_occurrence, {'min': 0, 'max': 1, 'palette': ['red', 'yellow', 'blue']}, "Water Mask")
Map.addLayerControl()
Map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

## Batch the export

In [17]:
def export_rois(roi_id, target_img, month_name):
    
    roi_name_temp = roi_id.split('/')[-1]
    roi_name = roi_name_temp.split('_roi')[0]
    feature = ee.FeatureCollection(roi_id).first()
    bounds = feature.geometry()

    clipped_img = target_img.clip(bounds)

    export_params = {
        'image': clipped_img,
        'description': f'GSWORecurrenceFullMonthly_{roi_name}_month_{month_name}_', 
        'fileNamePrefix': f'GSWORecurrenceFullMonthly_{roi_name}_month_{month_name}_',
        'folder': 'gswo_exports_full', 
        'scale': 30,
        'crs': 'EPSG:4326',
        'region': bounds,
        'fileFormat': 'GeoTIFF',
        'maxPixels': 1e13
    }

    ee.batch.Export.image.toDrive(**export_params).start()

In [18]:
for month, ds in monthly_datasets:
    for roi_id in rois_ids:
        export_rois(roi_id, ds, month)